# Solución etapa 4 - Training - Predict v2

In [ ]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl
from utils.explore_utils import explr

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect

from skforecast.model_selection import backtesting_forecaster
from skforecast.datasets import fetch_dataset
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import grid_search_forecaster


In [ ]:
version_to_load='v1'

In [ ]:
# Cargar el dataset
csv_path = f'../data/preprocessed/preprocessed_data_{version_to_load}.csv'
print('... Loading:', csv_path,'...')
final_data = pd.read_csv(csv_path)

tabl(final_data)

In [ ]:
final_data.info()

In [ ]:
target_column = 'Horas_Operativas'

In [ ]:
# Steps de predicción
steps = 15

## Funciones procesado

In [ ]:
# Preparar el dataset

def prep_equipo(equipo):
    equipo.drop(columns=['ID_Equipo','Tipo_Equipo','Modelo','Potencia_kW','Horas_Recomendadas_Revision','Fabricante'], inplace=True)

    equipo['Fecha'] = pd.to_datetime(equipo['Fecha'], format='%Y-%m-%d')

    # fecha como índice
    equipo = equipo.set_index('Fecha')

    # Eliminar índices duplicados (mantener el primero)
    equipo_clean = equipo[~equipo.index.duplicated(keep='first')]
    equipo_clean[equipo_clean.index.duplicated()]

    # Convierte Timeseries a frecuencia especificada.
    equipo_fq = equipo_clean.asfreq('D', fill_value=0)

    # print(equipo_fq.info())

    return equipo_fq


In [ ]:
# Separación datos train-test

def sep_train_test(equipo_fq, doPlot=True):
    steps = 15
    datos_train = equipo_fq[:-steps]
    datos_test  = equipo_fq[-steps:]
    # print(f"Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
    # print(f"Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")

    if(doPlot):
        fig, ax = plt.subplots(figsize=(16, 5))
        datos_train['Horas_Operativas'].plot(ax=ax, label='train')
        datos_test['Horas_Operativas'].plot(ax=ax, label='test')
        ax.legend()

    return datos_train, datos_test

In [ ]:
# Búsqueda de hiperparámetros: grid search

def search_best_forecaster(steps, lags_grid, param_grid, datos_train, exogs_train, target_column):
    forecaster = ForecasterRecursive(
        regressor=Ridge(random_state=123),
        transformer_y=StandardScaler(),
        lags=30
    )

    # Particiones de entrenamiento y validación
    cv = TimeSeriesFold(
        steps=steps,
        initial_train_size=int(len(datos_train) * 0.5),
        refit=False,
        fixed_train_size=False,
    )

    resultados_grid = grid_search_forecaster(
        forecaster=forecaster,
        y=datos_train[target_column],
        cv=cv,
        param_grid=param_grid,
        lags_grid=lags_grid,
        metric='mean_squared_error',
        return_best=True,
        n_jobs='auto',
        verbose=False,
        exog=exogs_train,
    )

    # Resultados de la búsqueda de hiperparámetros
    bestfc = resultados_grid.loc[0]

    return bestfc

In [ ]:
# Generar modelo final

def gen_finalModel(steps, best_lags, best_params, datos_train, exogs_train, target_column):
    # Crear y entrenar forecaster final
    forecaster = ForecasterRecursive(
        regressor=Ridge(alpha=best_params['alpha'], random_state=123),
        transformer_y=StandardScaler(),
        lags=best_lags
    )

    forecaster.fit(y=datos_train[target_column], exog=exogs_train)

    return forecaster

In [24]:
# Evaluar error

def get_error(target_column, predicciones, equipo_fq, datos_test):
    # Error test
    error_mse = mean_squared_error(
        y_true=datos_test[target_column],
        y_pred=predicciones
    )

    data_var = equipo_fq[target_column].var()

    mse2var = round(error_mse/data_var*100, 2)

    print(f"Error de test (mse): {error_mse}")
    print(f"Varianza datos: {data_var}")
    print(f"mse2var: {mse2var}%")

    return mse2var

## Un equipo

In [ ]:
eq = 1

In [ ]:
# Filtrar el df del equipo
equipo = final_data[final_data['ID_Equipo'] == eq].copy()

# Preparar el dataset para el equipo seleccionado
equipo_fq = prep_equipo(equipo)

In [ ]:
# Obtener datos train y test
datos_train, datos_test = sep_train_test(equipo_fq)

In [ ]:
# Valores candidatos de lags
lags_grid = [10, 50, 100, 150]

# Valores candidatos de hiperparámetros del regresor
param_grid = {
    'alpha': [0.1, 1, 10, 100],
}

# Exógenos para el modelo
exogs_train = datos_train.drop(columns=target_column)
exogs_test = datos_test.drop(columns=target_column)

In [ ]:
# Búsqueda de hiperparámetros
bestfc = search_best_forecaster(steps, lags_grid, param_grid, datos_train, exogs_train, target_column)
best_lags = len(bestfc.lags)
best_params = bestfc.params

print(best_lags, best_params)

In [ ]:
# modelo final
final_model = gen_finalModel(steps, best_lags, best_params, datos_train, exogs_train, target_column)

# Predicciones
predicciones = final_model.predict(steps=steps, exog=exogs_test)

# # Gráfico de predicciones vs valores reales
# fig, ax = plt.subplots(figsize=(16, 5))
# datos_train['Horas_Operativas'].plot(ax=ax, label='train')
# datos_test['Horas_Operativas'].plot(ax=ax, label='test')
# predicciones.plot(ax=ax, label='predicciones', color="red")
# ax.legend()

# Error test
get_error(target_column, predicciones, equipo_fq, datos_test)

## Iterar sobre equipos

In [27]:
# Valores candidatos de lags
lags_grid = [5, 10, 50, 100, 150]

# Valores candidatos de hiperparámetros del regresor
param_grid = {
    'alpha': [0.1, 1, 10, 100],
}

# resultados

resultados = {}


for eq in range(1, 7):
    print(f"********** Equipo {eq} ************")

    # Filtrar el df del equipo
    equipo = final_data[final_data['ID_Equipo'] == eq].copy()

    # Preparar el dataset para el equipo seleccionado
    equipo_fq = prep_equipo(equipo)

    # Obtener datos train y test
    datos_train, datos_test = sep_train_test(equipo_fq, False)

    # Exógenos para el modelo
    exogs_train = datos_train.drop(columns=target_column)
    exogs_test = datos_test.drop(columns=target_column)

    # Búsqueda de hiperparámetros
    bestfc = search_best_forecaster(steps, lags_grid, param_grid, datos_train, exogs_train, target_column)
    best_lags = len(bestfc.lags)
    best_params = bestfc.params

    # modelo final
    final_model = gen_finalModel(steps, best_lags, best_params, datos_train, exogs_train, target_column)

    # Predicciones
    predicciones = final_model.predict(steps=steps, exog=exogs_test)

    # Error test
    mse2var = get_error(target_column, predicciones, equipo_fq, datos_test)

    resultados[eq] = {
        'best_lags': best_lags,
        'best_params': best_params,
        'mse2var': mse2var
    }

    print('---'*20)

********** Equipo 1 ************


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] 
  Parameters: {'alpha': 0.1}
  Backtesting metric: 34976.66513452184
Error de test (mse): 20982.882283219362
Varianza datos: 101688.87860257094
mse2var: 20.63%
------------------------------------------------------------
********** Equipo 2 ************


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

...

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10] 
  Parameters: {'alpha': 0.1}
  Backtesting metric: 14545.578480724145
Error de test (mse): 9948.212417767383
Varianza datos: 58308.32094268328
mse2var: 17.06%
------------------------------------------------------------
********** Equipo 6 ************


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10] 
  Parameters: {'alpha': 1}
  Backtesting metric: 8607.01402505016
Error de test (mse): 54068.40575866353
Varianza datos: 55900.25571995162
mse2var: 96.72%
------------------------------------------------------------


# Resultados

In [26]:
resultados

{1: {'best_lags': 50, 'best_params': {'alpha': 0.1}, 'mse2var': 20.63},
 2: {'best_lags': 10, 'best_params': {'alpha': 1}, 'mse2var': 2.79},
 3: {'best_lags': 10, 'best_params': {'alpha': 0.1}, 'mse2var': 3.89},
 4: {'best_lags': 50, 'best_params': {'alpha': 0.1}, 'mse2var': 4.82},
 5: {'best_lags': 10, 'best_params': {'alpha': 0.1}, 'mse2var': 17.06},
 6: {'best_lags': 10, 'best_params': {'alpha': 1}, 'mse2var': 96.72},
 7: {'best_lags': 10, 'best_params': {'alpha': 0.1}, 'mse2var': 35.16},
 8: {'best_lags': 10, 'best_params': {'alpha': 10}, 'mse2var': 33.37},
 9: {'best_lags': 10, 'best_params': {'alpha': 10}, 'mse2var': 7.12},
 10: {'best_lags': 10, 'best_params': {'alpha': 10}, 'mse2var': 23.09},
 11: {'best_lags': 10, 'best_params': {'alpha': 0.1}, 'mse2var': 13.68},
 12: {'best_lags': 50, 'best_params': {'alpha': 0.1}, 'mse2var': 5.26},
 13: {'best_lags': 10, 'best_params': {'alpha': 0.1}, 'mse2var': 4.21},
 14: {'best_lags': 100, 'best_params': {'alpha': 10}, 'mse2var': 2.23}}